In [ ]:
from polymorph_s2df import *
import jax.numpy as jnp
import jax

SAMPLE_SIZE = 10000
INTEGRATION_BOUNDS = (-3, 3)
SEED = jax.random.PRNGKey(0)
points = jax.random.uniform(SEED, (SAMPLE_SIZE, 2), jnp.float32, *INTEGRATION_BOUNDS)

In [ ]:
################################
# Direct Optimistix optimization
# (not using our stuff)

def area(shape, points, bounds):
    samples = shape.is_inside(points)
    return samples.mean() * (bounds[1] - bounds[0])**2


import optimistix
from timeit import default_timer as timer
def optimize_params(cost, params, points):
    solver = optimistix.BFGS(rtol=1e-5, atol=1e-6)
    start = timer()
    solution = optimistix.minimise(cost, solver, params, points, throw=False)
    elapsed = timer() - start
    print("{0} steps in {1:.2f} seconds".format(
            solution.stats.get('num_steps'),
            elapsed))
    return solution.value

In [ ]:
import polymorph_num.expr as expr
from polymorph_num.expr import as_expr
from polymorph_num import loss, ops, optimizer, vec
from polymorph_num.ops import param

def v(x,y):
    return vec.Vec2(as_expr(x),as_expr(y))

apoints =v(points[:, 0], points[:, 1])

def area(f, points, bounds):
    distance = f(points)
    scale = as_expr(100)
    is_inside = as_expr(1) - (scale*distance).sigmoid()
    mean = expr.Sum(is_inside) / is_inside.dim
    return mean * (bounds[1] - bounds[0])**2


In [ ]:
#####################################
# Find radius for target circle area

r = param()
c = v(0.,0.)

def circle_sdf(radius, center, point):
    dx = center.x - point.x
    dy = center.y - point.y
    dist = (dx*dx + dy*dy).sqrt()
    return dist - radius


target_area = as_expr(3.141)
error = target_area - area(lambda p: circle_sdf(r, c, p), apoints, INTEGRATION_BOUNDS)

l = loss.Loss(error*error)
l.register_output(r)

opt = optimizer.Optimizer(l)
print("---")

def solve():
    start = timer()
    soln = opt.optimize({})
    elapsed = timer() - start
    print("optimized in {0:.2f} seconds".format(elapsed))
    return soln

soln = solve()
soln = solve()

print(soln.eval(r))

In [ ]:
p = param()
e1 = p + 1
e2 = (e1 - 3)**2
l = loss.Loss(e2)
l.register_output(p)
l.register_output(e1)
l.register_output(e2)
s = optimizer.Optimizer(l).optimize({})
print(s.eval(p))
print(s.eval(e1))
print(s.eval(e2))

In [ ]:
############################################
# Points that try to go to a given attractor

class DynamicPoint:
    attractor: vec.Vec2
    location: vec.Vec2

    def __init__(self, attractor: vec.Vec2):
        self.attractor = attractor
        self.location = vec.Vec2(param(), param())
        
    def loss(self):
        return (self.attractor - self.location).norm()

In [ ]:
####################################
# Single dynamic point is attracted

p = DynamicPoint(v(1., 2.))
l = loss.Loss(p.loss())
l.register_output(p.location.x)
l.register_output(p.location.y)

s = optimizer.Optimizer(l).optimize({})
print(s.eval(p.location.x), s.eval(p.location.y))

In [ ]:
# but can be constrained to move elsewhere...

target = v(5., 5.)
l = loss.Loss(p.loss() + (p.location - target).norm()*100)
l.register_output(p.location.x)
l.register_output(p.location.y)

s = optimizer.Optimizer(l).optimize({})
print(s.eval(p.location.x), s.eval(p.location.y))

In [ ]:
# Create points that are 3 apart from each other
a = DynamicPoint(v(0., 0.))
b = DynamicPoint(v(3., 0.))
points = [a, b]

# Add constraint that we want them to be 2 apart from each other
dist_target = 3.
dist_loss = ((a.location - b.location).norm() - dist_target)**2

# but also that they should stay near their initial positions when possible.
dp_loss = as_expr(0.)
for p in points:
    dp_loss += p.loss()

l = loss.Loss(dist_loss + dp_loss)
l.register_output(dp_loss)
l.register_output(dist_loss)
l.register_output(l.loss)

for p in points:
    l.register_output(p.location.x)
    l.register_output(p.location.y)

s = optimizer.Optimizer(l).optimize({})

for p in points:
    print(s.eval(p.location.x), s.eval(p.location.y))

print(s.eval(dp_loss))
print(s.eval(dist_loss))

# TODO: Seems like the solver isn't working in this example.
# It leaves the points at 0,0 even though the losses are positive.


In [ ]:
####################################
# Axis-aligned right triangle (WIP)
# a.
# ....
# .  ...
# .    ...
# c.......b
#
# first create points, drawn with a bit of error
import random
random.seed(1)
err = lambda: 0.2*random.random()
side = 1.0

a = v( param() + err()
      ,param() + side + err())
b = v( param() + side + err()
      ,param() + err())
c = v( param() + err()
      ,param() + err())


# solve for horizontal and vertical constraints
def err_horizontal(a,b):
    return a.y - b.y
def err_vertical(a,b):
    return a.x - b.x

l = loss.Loss(err_horizontal(c, b)**2 + err_vertical(a, c)**2)
l.register_output(l.loss)

# for p in [a, b, c]:
#     l.register_output(p.x)
#     l.register_output(p.y)

solution = optimizer.Optimizer(l).optimize({})

